# How the Ren'Py Auto-Translator Works ⚙️

This notebook automates the translation of Ren'Py (`.rpy`) script files by using large language models (LLMs). It's designed to read your game's text, send it to an AI for translation, and then reassemble it into a new, translated `.rpy` file while preserving the original code structure and formatting.

---

## 1. Setup and Configuration

First, the notebook sets up the environment. You provide **API keys** for the API you want to use (like Google AI Studio or Google Vertex (Not tested) or an OpenAI-compatible APIs).

Next, you configure the translation by defining:
* **Source and Target Languages**: You specify the original language of the script and the language you want to translate it into (e.g., "English" to "Persian").
* **Prompts**:
    * The **System Prompt** acts as the main instruction manual for the AI. It tells the model its role (an expert Ren'Py translator) and gives it strict rules to follow, such as preserving code, formatting tags (`{b}`, `\n`), and special characters.
    * The **Game-Specific Prompt** is where you provide context about your game. This includes character names, desired tone, and translations for recurring terms, which helps the AI produce a more accurate and consistent translation.

---

## 2. Processing and Chunking

Once you provide your input `.rpy` file, the notebook doesn't send the entire file to the AI at once. Large files can exceed the model's context limit.

Instead, it performs two key steps:
1.  **Preprocessing**: It reads your file and identifies all the translatable blocks of text (dialogue and UI elements).
2.  **Chunking**: It intelligently groups these blocks into smaller **chunks**. Each chunk is sized to fit within the AI model's maximum input token limit, ensuring that no request is too large for the model to handle.

---

## 3. The Translation Loop

The core of the notebook is the translation loop, which processes one chunk at a time:

1.  **API Call**: It sends a chunk of your game's text to the selected AI model, along with the detailed prompts you configured.
2.  **AI Translation**: The AI follows the instructions to translate only the dialogue and UI text within the chunk, leaving all code, comments, and formatting tags intact.
3.  **Validation**: After receiving the translation, the notebook performs a quick check to ensure the output is valid. It verifies that the number of translation blocks is correct and that the structure hasn't been corrupted.
4.  **Retry on Failure**: If a translation fails validation or if there's an API error, the notebook will automatically **retry** the request a few times. It may slightly increase the "temperature" (randomness) on subsequent tries to get a better result.

This process repeats until all chunks have been successfully translated.

---

## 4. Final Output

After the loop finishes, the notebook takes all the translated chunks and stitches them back together in the correct order. The final, complete translation is then saved to a new `.rpy` file, which is named using the target language (e.g., `input_file_Persian.rpy`). You can then download this file and use it in your Ren'Py project.

# Setup

In [ ]:
!pip install -q datasets

In [ ]:
#@title Set API Keys here
# from google.colab import userdata
# API_KEY = userdata.get("api_key")

GEMINI_API_KEY = "" #@param {"type": "string"}
OPENAI_API_KEY = "" #@param {"type": "string"}

## Either run **Gemini with Google Client**, or **OpenAI-compatible API**.

## Google API

In [ ]:
#@title Define Google Client (NO JSON SCHEMA SUPPORT YET)
from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)

from google.genai import types

GOOGLE_MODEL_NAME = "gemini-2.0-flash" # @param ["gemini-2.5-pro","gemini-2.5-flash","gemini-2.0-flash"] {"allow-input":true}
MAX_OUTPUT_TOKENS = 6144 #@param {"type": "integer"}

google_safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    )
]

def create_completion_with_client(user_prompt, system_prompt, temperature=0.2):
    completion = client.models.generate_content(
        model=GOOGLE_MODEL_NAME,
        contents=[user_prompt],
        config=types.GenerateContentConfig(
            system_instruction=system_prompt,
            temperature=temperature,
            max_output_tokens=MAX_OUTPUT_TOKENS,
            safety_settings=google_safety_settings,
        ),
    )
    return completion.text

## OpenAI-Compatible API

In [ ]:
#@title Define OpenAI Client
from openai import OpenAI

API_URL = "https://openrouter.ai/api/v1" #@param {"type": "string", "allow-input": true} ["https://openrouter.ai/api/v1", "https://api.cohere.ai/compatibility/v1"]
MODEL_NAME = "meta-llama/llama-4-maverick:free:nitro" # @param ["meta-llama/llama-4-maverick:free:nitro","command-r-plus-04-2024","command-r7b-12-2024","command-r-08-2024","command-a-03-2025"] {"allow-input":true}
MAX_OUTPUT_TOKENS = 8192 #@param {"type": "integer"}

client = OpenAI(
    api_key= OPENAI_API_KEY,
    base_url= API_URL,
)

my_safety_settings = [
    # {"category": "HARM_CATEGORY_UNSPECIFIED", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_UNSPECIFIED", "threshold": "OFF"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "OFF"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "OFF"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "OFF"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "OFF"},
    {"category": "HARM_CATEGORY_CIVIC_INTEGRITY", "threshold": "OFF"},
]
google_safety_settings = {
    "safety_settings": my_safety_settings
}

def is_google_model(model_name):
    return "google/" in model_name.lower() or 'gemini' in model_name.lower()

def create_completion_with_client(user_prompt, system_prompt, json_schema=None, extra_body=None, temperature=0.2):
    try:
        completion = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=temperature,
            max_tokens=MAX_OUTPUT_TOKENS,
            response_format = { "type": "json_object", "schema": json_schema } if json_schema is not None else None,
            stream=False,
            extra_body=extra_body if extra_body is not None else google_safety_settings if is_google_model(MODEL_NAME) else None
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error during API call: {e}")
        return None

# Prompts and values

In [ ]:
#@title Set Languages and SYSTEM PROMPT
#@markdown Source and Target languages, should be exactly the same as the ones used in Renpy, and `.rpy` translation files.
SOURCE_LANGUAGE = "English" #@param {"type": "string", "allow-input": true}
TARGET_LANGUAGE = "Persian" #@param {"type": "string", "allow-input": true}
#@markdown This general system prompt is good enough for most use cases, but feel free to change it.
SYSTEM_PROMPT = """You are an expert AI translator specializing in translating visual novel games from [SRC_LANG] to [TGT_LANG].
Your task is to process the provided Ren'Py translation segment and output the complete segment with the [SRC_LANG] strings translated into [TGT_LANG].

**Input Format:**
The file contains two types of translation blocks:

1.  **Dialogue Lines:**
    These blocks start with `translate [TGT_LANG] label_identifier:`. They sometimes include CHAR_NAME which may look like this: mc, Teller, etc.
    Example:
    translate [TGT_LANG] label_identifier:

        CHAR_NAME "dialogue line with {b}tags{/b} and \n newlines."
    For these blocks, you must translate **only** the string.

2.  **UI/Menu Strings (under `translate [TGT_LANG] strings:`):**
    These blocks appear under a line `translate [TGT_LANG] strings:`.
    Example:
    old "[SRC_LANG] UI string (with context)."
    new "[SRC_LANG] UI string (with context)."
    For these blocks, you must translate **only** the string following the `new` keyword.

**Key Instructions:**
1.  **Target Language:** For this task, Target Language is [TGT_LANG]. The Source Language is [SRC_LANG].
2.  **Accurate and Natural Translation:** Translate the text accurately and naturally into Persian, ensuring the translation is appropriate for the context.
3.  **Preserve Structure and Formatting:**
    *   You MUST preserve the entire original structure, including any comments  (lines starting with `#`) if included, `translate` lines, `old` lines, keywords (like `translate`, `old`, `new`), colons, quotation marks, and existing indentation.
    *   Your output should be the complete segment, modified only with the translated text in the designated places.
4.  **Handle Special Characters and Tags:**
    *   **Newline Characters (`\n`):** These must be preserved and placed correctly within the translated Persian string to reflect the original line breaks.
    *   **Ren'Py Formatting Tags (e.g., `{b}`, `{/b}`, `{i}`, `{/i}`):** These tags must be preserved. In the translated string, they should surround the Persian equivalent of the text that was originally tagged in [SRC_LANG]. For example, if "It's {b}YOU{/b} who should enjoy" is the source, the Persian translation should include `{b}` and `{/b}` around the translation of "YOU".
    *   **Percentage Signs (`%%`):** Preserve these as they appear (e.g., `50%%`). If Persian has a different convention for percentages in this context, apply the correct Persian formatting.
    *   **Other Game-Specific Placeholders:** If you encounter any other placeholders (e.g., `[player_name]`), they should generally be kept as-is unless the context implies they represent a translatable concept.
5.  **Contextual Clues in Parentheses:**
    *   Text within parentheses, such as `(This will be your name)`, `(Sincere)`, `(Unlocks Athletic trait)`, or `(Change name)`, often provides important context or describes an action/choice. This text **must also be translated** into Persian.
6.  **Consistency:** Maintain consistent terminology for repeated game concepts or interface elements throughout the translation.

**Output:**
Your output should be the complete Ren'Py translation segment, with only the specified [SRC_LANG] strings replaced by their [TGT_LANG] (Persian) translations. Do not add any conversational text, introductions, or explanations outside of the Ren'Py file structure itself.

[EXAMPLES]

Now, please process the input I provide according to these instructions.
"""

#@markdown Also provide your own Examples in here.
EXAMPLES = """**Example of processing a dialogue line:**
Source Input:
translate Persian start_d6b3d5a6:

    # "If you prefer to go in blind without hints such as these it's ok too.\nIt's {b}YOU{/b} who should enjoy the game after all."
    "If you prefer to go in blind without hints such as these it's ok too.\nIt's {b}YOU{/b} who should enjoy the game after all."

Expected Persian Output:
translate Persian start_d6b3d5a6:

    # "If you prefer to go in blind without hints such as these it's ok too.\nIt's {b}YOU{/b} who should enjoy the game after all."
    "اگر ترجیح می‌دهید بدون راهنمایی‌هایی از این قبیل به صورت کورکورانه پیش بروید نیز اشکالی ندارد.\nدر نهایت این {b}شما{/b} هستید که باید از بازی لذت ببرید."


**Example of processing a UI/Menu string:**
Source Input:
    # game/script.rpy:494
    old "Yes, speaking. (This will be your name)"
    new "Yes, speaking. (This will be your name)"

Expected Persian Output:
    # game/script.rpy:494
    old "Yes, speaking. (This will be your name)"
    new "بله، خودم هستم. (اسم شما این خواهد بود)"
"""
SYSTEM_PROMPT = SYSTEM_PROMPT.replace("[SRC_LANG]", SOURCE_LANGUAGE)
SYSTEM_PROMPT = SYSTEM_PROMPT.replace("[TGT_LANG]", TARGET_LANGUAGE)
SYSTEM_PROMPT = SYSTEM_PROMPT.replace("[EXAMPLES]", EXAMPLES)

### How to Create a Game-Specific System Prompt

This guide helps you generate a high-quality `GAME_SPECIFIC_PROMPT` by using an LLM to analyze your game's script.

#### Step 1: Gather Game Scripts

Collect the text that the AI will analyze.

*   **For small games:** If your project isn't too large, you can give all your `.rpy` script files directly to an LLM model.
*   **For larger games:** To avoid exceeding the AI's context limit, select representative samples:
    *   Dialogue-heavy scripts to capture character voices.
    *   Narration and main character (mc) thoughts to define the game's tone.
    *   UI text from files like `screens.rpy`.

#### Step 2: Generate the Guide with a Meta-Prompt

Use the template below to instruct a model with larger context length (like Gemini 2.5 Pro) to create your guide.

**Copy and paste this into the AI:**

```text
Analyze the provided Ren'Py script and write a "Translation Guide" for translating from [Source Language] to [Target Language]. The guide must be a structured Markdown file with the following sections:

1.  **Game Overview & Tone:** General tone, narrator's style, and formality level.
2.  **Character Guide:** A table listing each character's name, personality, and speaking style.
3.  **Glossary of Key Terms:** A table of recurring game-specific terms, locations, or items.
4.  **UI/Menu Text Style:** A brief description of the style for UI text.
```

**Before running:**
*   Replace `[Source Language]` and `[Target Language]`.
*   Paste your collected script from Step 1 at the end.

#### Step 3: Review and Finalize

The AI's output is a **draft**, not a final product. Your review is essential.

*   **Correct** any errors in the AI's analysis of tone or characters.
*   **Add** any missing terms, characters, or context you know is important.
*   **Fill in** any suggested translations in the glossary.
*   **Clean up** and remove any irrelevant text.

The final, edited text is your `GAME_SPECIFIC_PROMPT`. Copy it directly into the Colab notebook.

In [ ]:
#@title GAME SPECIFIC PROMPT
#@markdown Enter your game-specific system prompt here.
GAME_SPECIFIC_PROMPT = """## "My Awesome Game" - Translation Guide

**Game Overview & Tone:**
*   **General Tone:** A slice-of-life high school drama with comedic elements. The tone is generally informal and modern.
*   **Dialogues:** Use informal language for conversations between students. Use formal language when students talk to teachers.
*   **Narrator (mc's thoughts):** The narration is introspective, personal, and sometimes sarcastic. The translation should reflect this inner monologue.

**Character Guide:**

| Character Name/ID | Personality & Role                        | Speaking Style                                       |
| :---------------- | :---------------------------------------- | :--------------------------------------------------- |
| `mc` / `[name]`   | The protagonist. Can be witty or sincere. | Varies by player choice, but generally modern and direct. |
| `jess`            | The energetic and cheerful best friend.   | Uses a lot of slang, exclamation marks, and is very expressive. |
| `mr_harrison`     | The strict but fair history teacher.      | Speaks formally, uses complex vocabulary. Always polite. |
| `???`             | A mysterious character.                   | Speaks in short, cryptic sentences.                  |

**Glossary of Key Terms:**

| English Term             | Target Translation | Notes                                      |
| :----------------------- | :------------------ | :----------------------------------------- |
| The Sunstone             | The Sunstone        | A key item in the story. Keep in English.  |
| Everwood High            | Everwood High       | The name of the school.                    |
| The "Glimmer"            | The "Glimmer"       | A special power some characters have.      |

**UI/Menu Text Style:**
*   UI text should be clear and concise. For example, "Settings," "Save Game," "Load Game."

**Cultural & Idiomatic Notes:**
*   The phrase "break a leg" appears before a performance. This should be translated to its local equivalent for "good luck."
*   Jokes often rely on English puns. These should be localized to be funny in the target language, even if it means changing the joke."""

In [ ]:
#@title Get System prompt function
def get_system_prompt():
    return SYSTEM_PROMPT + "\n" + GAME_SPECIFIC_PROMPT

In [ ]:
#@title Get Character Per Token
#@markdown Add your own language here, depending on the **language**, and chosen **model**, these values changes, but default values are good estimates for English and Persian for OpenAI, Gemini and Cohere models.
#@markdown <br> To obtain this, you can use online tokenizers for your model of choice. <br> For Google models, you can use **Google AI Studio** to get the token count of an input text.
def get_char_per_token(lang):
    if lang == "English":
        return 3.75
    elif lang == "Persian":
        return 2.8

In [ ]:
#@title Set retries and temperature
RETRIES_NUM = 3 #@param {"type": "integer"}
DEFAULT_TEMPERATURE = 0.7 # @param {"type":"slider","min":0,"max":2,"step":0.05}
def get_temperature(tries):
    #@markdown You can also adjust temperature based on number of failed tries, for example increase temperature
    return DEFAULT_TEMPERATURE

# Function definitions

In [ ]:
#@title Blocks and Chunks functions
def get_blocks_from_text(text, trg_lng):
    blocks = text.split(f"translate {trg_lng} ")
    return [f"translate {trg_lng} " + block for block in blocks]

def get_input_token_count(text):
    return len(text) // get_char_per_token(SOURCE_LANGUAGE)

def get_string_from_chunk(chunk):
    return "\n".join(chunk)

def get_chunks_to_translate(blocks_to_translate, char_per_token, max_input_token):
    if get_input_token_count(get_string_from_chunk(blocks_to_translate)) < max_input_token:
        return [blocks_to_translate]

    all_chunks = []
    current_idx = 0
    while current_idx < len(blocks_to_translate):
        current_chunk = []
        current_size = 0
        while current_idx < len(blocks_to_translate) and current_size < max_input_token:
            current_chunk.append(blocks_to_translate[current_idx])
            current_size += get_input_token_count(blocks_to_translate[current_idx])
            current_idx += 1
        all_chunks.append(current_chunk)
    return all_chunks

def get_estimated_input_tokens(MAX_OUTPUT_TOKENS):
    # Estimate input tokens based on max output tokens
    return MAX_OUTPUT_TOKENS * get_char_per_token(TARGET_LANGUAGE) // (1.2 * get_char_per_token(SOURCE_LANGUAGE))

In [ ]:
#@title Replace and Restore Hashes in Translation
def replace_hashes_with_placeholders(text, trg_lng):
    hash_map = {}
    placeholder_counter = 0

    BLOCK_ID_REGEX = rf"(translate {trg_lng} )(\w+):"
    def replacer(match):
        nonlocal placeholder_counter
        placeholder = f"_HASH_{placeholder_counter}"
        hash_map[placeholder] = match.group(2)
        placeholder_counter += 1
        return f"{match.group(1)}{placeholder}"

    text_with_placeholders = re.sub(BLOCK_ID_REGEX, replacer, text)
    return text_with_placeholders, hash_map

def restore_hashes_from_placeholders(text_with_placeholders, hash_map):
    restored_text = text_with_placeholders
    # Iterate through the map and replace each placeholder with its original hash
    # Replacing longest placeholders first can prevent issues if one placeholder is a substring of another,
    # though our __HASH__<number>__ format avoids this. Still, sorting is safer.
    for placeholder in sorted(hash_map.keys(), key=len, reverse=True):
        original_hash = hash_map[placeholder]
        restored_text = restored_text.replace(placeholder, original_hash)
    return restored_text

In [ ]:
#@title Translation Validation functions
def is_strings_block_valid(chunk_str, translation):
    OLD_NEW_REGEX = r'^\s*old\s+"(?:[^"\\]|\\.)*"\s+^\s*new\s+"(?:[^"\\]|\\.)*"'
    return len(re.findall(OLD_NEW_REGEX, chunk_str, re.MULTILINE)) == len(re.findall(OLD_NEW_REGEX, translation, re.MULTILINE))

BLOCK_ID_REGEX = rf"translate {TARGET_LANGUAGE} (\w+):"
def is_translation_valid(chunk_str, translation):
    chunk_blocks = get_blocks_from_text(chunk_str, TARGET_LANGUAGE)
    translation_blocks = get_blocks_from_text(translation, TARGET_LANGUAGE)
    if len(chunk_blocks) != len(translation_blocks):
        print(f"Translation Invalid: Original has {len(chunk_blocks)} blocks, but translated has {len(translation_blocks)}")
        return False

    chunk_block_ids = set(re.findall(BLOCK_ID_REGEX, "".join(chunk_blocks)))
    translation_block_ids = set(re.findall(BLOCK_ID_REGEX, "".join(translation_blocks)))
    if chunk_block_ids != translation_block_ids:
        missing_ids = chunk_block_ids - translation_block_ids
        extra_ids = translation_block_ids - chunk_block_ids
        error_msg = "Translation Invalid: Block IDs mismatch."
        if missing_ids:
             error_msg += f" Missing IDs: {missing_ids}."
        if extra_ids:
             error_msg += f" Extra IDs: {extra_ids}."
        print(error_msg)
        return False

    for block in translation_blocks:
        content_after_header = block.split(':', 1)[-1] if ':' in block else block
        if content_after_header.count('"') % 2 != 0:
            print(f"Translation Invalid: Unmatched quotes found in block starting with:\n{block.splitlines()[0]}")
            return False

    if "strings" in chunk_block_ids:
        strings_chunk_block = [block for block in chunk_blocks if f"translate {TARGET_LANGUAGE} strings:" in block][0]
        strings_translation_block = [block for block in translation_blocks if f"translate {TARGET_LANGUAGE} strings:" in block][0]
        if not is_strings_block_valid(strings_chunk_block, strings_translation_block):
             print(f"Translation Invalid: 'strings' block old/new pair count mismatch.")
             return False

    return True

# Translation Post-Processing
def postprocess_translation(translation):
    # Sometimes the model outputs in ```renpy ... ``` format, so remove it
    if translation.count("```") % 2 != 0:
        return translation
    if "```renpy" in translation:
        return translation[translation.find("```renpy") + len("```renpy"):translation.rfind("```")].strip()
    if "```" in translation:
        return translation[translation.find("```") + len("```"):translation.rfind("```")].strip()
    return translation

# Process each file

In [ ]:
#@title Input file and Preprocess
#@markdown Upload your files, and input the file name you want to process
INPUT_FILE = "input_file.rpy" #@param {"type": "string"}
orig_text = open(INPUT_FILE, "r").readlines()

import re
# Remove comment lines
COMMENT_REGEX = r"^\s*#.*$"
orig_text = [line for line in orig_text if not re.match(COMMENT_REGEX, line)]

# Join all lines into a single string
orig_text = "".join(orig_text)

blocks_to_translate = get_blocks_from_text(orig_text, TARGET_LANGUAGE)[1:]

In [ ]:
#@title Calculate max input length and Turn into chunks
max_input_tokens = get_estimated_input_tokens(MAX_OUTPUT_TOKENS)
print(f"Max input tokens per chunk: {max_input_tokens}")

chunks_to_translate = get_chunks_to_translate(blocks_to_translate, get_char_per_token(SOURCE_LANGUAGE), max_input_tokens)
print(f"Total number of chunks: {len(chunks_to_translate)}")
print(f"Total number of (estimated) tokens: {sum([get_input_token_count(get_string_from_chunk(chunk)) for chunk in chunks_to_translate])}")

In [ ]:
#@title Reset translated chunks list (Re-run to reset the chunks translation progress)
translated_chunks = []

In [ ]:
#@title Translate loop
from tqdm import tqdm
#@markdown You can use this variable to continue from a chunk if you stopped for any reason
from_index = 0 #@param {type:"integer"}

for i in tqdm(range(from_index, len(chunks_to_translate))):
    try:
        chunk = chunks_to_translate[i]
        chunk_str = get_string_from_chunk(chunk)
        chunk_str_processed, hash_map = replace_hashes_with_placeholders(chunk_str, TARGET_LANGUAGE)

        tries = 0
        system_prompt = get_system_prompt()
        while tries < RETRIES_NUM:
            temperature = get_temperature(tries)
            translation_with_placeholders = create_completion_with_client(chunk_str_processed, system_prompt, temperature=temperature)
            if translation_with_placeholders is None:
                print(f"Translation for chunk {i} failed at attempt {tries + 1}/{RETRIES_NUM}: API call returned None.")
                tries += 1
                continue
            translated_str = restore_hashes_from_placeholders(translation_with_placeholders, hash_map)
            translated_str = postprocess_translation(translated_str)
            if not is_translation_valid(chunk_str, translated_str):
                print(f"Translation for chunk {i} failed validation at attempt {tries + 1}/{RETRIES_NUM}.")
                print("\nRestored translation (validation failed):")
                print(translated_str[:500] + "\n...\n" + translated_str[-500:])
                print("-" * 100) # Separator
                tries += 1
                continue
            print(f"Translated chunk {i}.")
            print(translated_str[:500] + "\n...\n" + translated_str[-500:])
            print("-" * 100)
            translated_chunks.append(translated_str)
            break
        else:
            print(f"Translation for chunk {i} failed after {RETRIES_NUM} tries.")
            break
    except Exception as e:
        print(f"Error during API call: {e}")

In [ ]:
#@title Did all chunks translate?
print(f"{len(translated_chunks)}/{len(chunks_to_translate)}")
if len(translated_chunks) < len(chunks_to_translate):
    print("TRANSLATION FAILED!")
else:
    print("TRANSLATION S")

In [ ]:
#@title Save to file
OUTPUT_FILE = INPUT_FILE.replace(".rpy", f"_{TARGET_LANGUAGE}.rpy")

with open(OUTPUT_FILE, "w") as f:
    for chunk in translated_chunks:
        f.write(chunk.strip() + "\n")